In [4]:
import matplotlib.pyplot as plt
from sksurv.linear_model import CoxPHSurvivalAnalysis
import numpy as np
import pandas as pd
import pkgutil
import io
import sys
sys.path.append('../')
sys.path.append('../auton-survival/')

In [7]:
def read_seer_csv(csv_file, drop_cols=None):
    """Helper function to load and preprocess the SEER dataset.

    """
    data = pkgutil.get_data(__name__, csv_file)
    data = pd.read_csv(io.BytesIO(data), low_memory=False)

    if drop_cols is not None:
        if 'Patient ID' in data.columns:
            drop_cols.append('Patient ID')
        data = data.drop(drop_cols, axis=1)
    else:
        if 'Patient ID' in data.columns:
            drop_cols = ['Patient ID']
            data.drop(drop_cols, axis=1, inplace=True)

    return data

In [11]:
x = read_seer_csv('datasets/ConditionalER_2_breast condition_ga_modified.csv',drop_cols=['State'])

In [12]:
col_event = x['death'].map({0:False,1:True})
col_time = x['d.time']
y = np.empty(dtype=[('col_event', bool), ('col_time', np.float64)], shape=x.shape[0])

y['col_event'] = col_event.values
y['col_time'] = col_time.values

x.drop(['d.time', 'death'], axis=1,inplace=True)
x_one_h = pd.get_dummies(x)



In [13]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x_one_h,y,test_size=0.2,random_state=101)

In [ ]:
estimator = CoxPHSurvivalAnalysis(alpha=0.0001).fit(x_train,y_train)
print(estimator.score(x_train,y_train))
estimator.score(x_test,y_test)